# Modele RESEAUX DE NEURONES 1

## Importataion

In [16]:
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

In [8]:
data=pd.read_csv('train_set.csv')
data=data.drop(['moyeneGeneral','Note1', 'Note2', 'Note3','Note4', 'Note5', 'Note6', 'Note7', 'Note8','Nom','ID'],axis=1)
data.head()

,Unnamed: 0,"Serie,x",GroupeAnonymat,LieuNaissance,Centre,Etablissement,Decision,Willaya,moughataa,DateNaissance
0,27634,SN,SN09,Bathet meit,Lycée Jedida,Lycée Jedida,Ajourné,Nouakchott 2 (Ouest),KSAR,12 ديسمبر 1998
1,36311,LO,LO1,Beir Taouress,Lycée Arafat 2 (A),Lycée Arafat 2,Ajourné,Nouakchott 3 (Sud),ARAFAT,26 ديسمبر 1991
2,21059,LM,LM3,Akjoujt,Lycée Akjoujt,Lycée Akjoujt,Ajourné,Inchiri,AKJOUJT,8 مايو 1995
3,24532,SN,SN06,Chelkhet Tiyab,Lycée Toujounine 1,Lycée Toujounine 1,Ajourné,Nouakchott 1 (Nord),TOUJOUNINE,24 avr. 1999
4,17861,SN,SN03,Aere M'Bar,Lycée Nouadhibou 2,Lycée Wava Nouadhibou,Ajourné,Dakhlet NDB,NOUADHIBOU,10 mai 2001


## preprocessing

In [9]:
### creer une colonne pour connaitre l'age
data['age'] = data['DateNaissance'].apply(  
    lambda x: int(x.split("-")[2] if '-' in x else int(x.split(" ")[2]) )
    ).apply(lambda x : 2021 - x if x > 1900  else ( 121 - x if x > 21 else 21 - x))

In [10]:
data=data.drop('Unnamed: 0',axis=1)

In [11]:
# Transformer les observations
data['Decision'] = data['Decision'].replace(['Ajourné', 'Sessionnaire', 'Abscent', 'Examen annulé à cause du Téléphone', 'Examen annulé à cause du comportement'], 'Ajourné')

In [12]:
### encoder la variable target 
### pour l'encodage on a associe 0 a admis et 1 pour ajournee 
data['Decision']=pd.get_dummies(data['Decision'],drop_first=True)
data.head()

,"Serie,x",GroupeAnonymat,LieuNaissance,Centre,Etablissement,Decision,Willaya,moughataa,DateNaissance,age
0,SN,SN09,Bathet meit,Lycée Jedida,Lycée Jedida,1,Nouakchott 2 (Ouest),KSAR,12 ديسمبر 1998,23
1,LO,LO1,Beir Taouress,Lycée Arafat 2 (A),Lycée Arafat 2,1,Nouakchott 3 (Sud),ARAFAT,26 ديسمبر 1991,30
2,LM,LM3,Akjoujt,Lycée Akjoujt,Lycée Akjoujt,1,Inchiri,AKJOUJT,8 مايو 1995,26
3,SN,SN06,Chelkhet Tiyab,Lycée Toujounine 1,Lycée Toujounine 1,1,Nouakchott 1 (Nord),TOUJOUNINE,24 avr. 1999,22
4,SN,SN03,Aere M'Bar,Lycée Nouadhibou 2,Lycée Wava Nouadhibou,1,Dakhlet NDB,NOUADHIBOU,10 mai 2001,20


In [108]:
# Séparer les variables indépendantes et la variable dépendante
X = data.drop('Decision',axis=1)
y = data['Decision']

In [109]:
# Diviser les données en données d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42,stratify =y)

In [110]:
# Définir les colonnes catégorielles et numériques
cat_cols = ['Serie,x', 'GroupeAnonymat', 'LieuNaissance', 'Centre', 'Etablissement', 'Willaya', 'moughataa']
num_cols = ['age']

In [111]:
# Prétraitement des données
preprocessor = ColumnTransformer(transformers=[
    ('num', StandardScaler(), num_cols),
    ('cat', OneHotEncoder(handle_unknown='ignore'), cat_cols)])

X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

In [112]:
X_train.shape

(19115, 1061)

## Entrainer le modele 

On va construire un réseaux de neurones de type MLP en utilisant la fonction MLPClassifier() . Ici on construit un MLP de 13-13-13, c.a.d 13 entrées, 13 neurones dans la première couche cachée et 13 neurones dans la deuxième couche cachée, avec un maximum d’itération (époques) égale à 700

In [71]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(hidden_layer_sizes=(10,13, 13) , max_iter=700)

In [72]:
mlp.fit(X_train,y_train)

MLPClassifier(hidden_layer_sizes=(10, 13, 13), max_iter=700)

In [24]:
pred=mlp.predict(X_test)

In [73]:
pred

array([1, 0, 1, ..., 1, 1, 1], dtype=uint8)

### Evaluation du modele

In [27]:
from sklearn.metrics import classification_report, confusion_matrix

In [30]:
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.32      0.19      0.24       395
           1       0.93      0.96      0.95      4384

    accuracy                           0.90      4779
   macro avg       0.62      0.58      0.59      4779
weighted avg       0.88      0.90      0.89      4779



### Prediction du dataset submission_template

In [31]:
### Importation
new=pd.read_csv('new_data.csv')
new=new.drop('Unnamed: 0',axis=1)
new=new.drop('prediction',axis=1)

In [32]:
new1=new.drop('Nom',axis=1)

In [33]:
X_test2 = preprocessor.transform(new1)

In [35]:
y_pred2 = mlp.predict(X_test2)

In [36]:
g=pd.DataFrame(y_pred2)
g.value_counts()

1    15018
0      912
dtype: int64

In [37]:
proba=mlp.predict_proba(X_test2)

In [38]:
pre=proba[:, 0]
len(pre)

15930

In [39]:
new1.head()

,ID,"Serie,x",GroupeAnonymat,LieuNaissance,Centre,Etablissement,Willaya,moughataa,age
0,ID_15,SN,SN09,Vassala,Lycée Vassala,Lycée Vassala,Hod Charghy,BASSEKNOU,22
1,ID_22,SN,SN09,Vassala,Lycée Vassala,Lycée Vassala,Hod Charghy,BASSEKNOU,18
2,ID_23,SN,SN09,Vassala,Lycée Vassala,Lycée Vassala,Hod Charghy,BASSEKNOU,18
3,ID_29,SN,SN09,Vassala,Lycée Vassala,Lycée Vassala,Hod Charghy,BASSEKNOU,21
4,ID_34,SN,SN09,Vassala,Lycée Vassala,Lycée Vassala,Hod Charghy,BASSEKNOU,21


In [40]:
new1['predicted2']=pre

In [41]:
submission=new1[['ID','predicted2']]

In [42]:
submission

,ID,predicted2
0,ID_15,6.465927e-09
1,ID_22,6.434701e-05
2,ID_23,6.434701e-05
3,ID_29,6.322630e-08
4,ID_34,6.322630e-08
...,...,...
15925,ID_13569,7.514444e-03
15926,ID_31670,8.427066e-10
15927,ID_31759,9.038548e-12
15928,ID_5855,2.504802e-06


In [43]:
submission.to_csv('Rn1.csv',index=False)

# Modele RESEAUX DE NEURONES 2

In [145]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense 

In [146]:
import numpy as np
from scipy.sparse import csr_matrix

# Convertir les tenseurs creux en tenseurs denses
X_train_dense = np.array(X_train.todense())
y_train_dense = np.array(y_train)

# Définir le modèle
model = Sequential()
model.add(Dense(64, input_dim=X_train_dense.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compiler le modèle
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Entraîner le modèle
model.fit(X_train_dense, y_train_dense, epochs=10, batch_size=32)

Epoch 1/10
598/598 [==============================] - 3s 3ms/step - loss: 0.2606 - accuracy: 0.9153
Epoch 2/10
598/598 [==============================] - 2s 4ms/step - loss: 0.2253 - accuracy: 0.9211
Epoch 3/10
598/598 [==============================] - 2s 4ms/step - loss: 0.2166 - accuracy: 0.9223
Epoch 4/10
598/598 [==============================] - 2s 3ms/step - loss: 0.2102 - accuracy: 0.9259
Epoch 5/10
598/598 [==============================] - 2s 3ms/step - loss: 0.2024 - accuracy: 0.9263
Epoch 6/10
598/598 [==============================] - 2s 3ms/step - loss: 0.1959 - accuracy: 0.9288
Epoch 7/10
598/598 [==============================] - 2s 3ms/step - loss: 0.1880 - accuracy: 0.9310
Epoch 8/10
598/598 [==============================] - 2s 3ms/step - loss: 0.1800 - accuracy: 0.9331
Epoch 9/10
598/598 [==============================] - 2s 3ms/step - loss: 0.1727 - accuracy: 0.9341
Epoch 10/10
598/598 [==============================] - 2s 4ms/step - loss: 0.1652 - accuracy: 0.9370

### Commentaire:

ce  modèle est un modele de réseau de neurones est ensuite défini à l'aide de la classe Sequential de Keras, qui permet de définir des modèles de réseaux de neurones en empilant des couches les unes sur les autres. Le modèle est constitué de trois couches denses. La première couche est une couche dense de 64 neurones avec une fonction d'activation ReLU. La deuxième couche est une couche dense de 32 neurones avec la même fonction d'activation. Enfin, la troisième couche est une couche dense de sortie de 1 neurone avec une fonction d'activation sigmoid.

L'entraînement du modèle est effectué avec la méthode fit() en fournissant les données d'entraînement X_train_dense et y_train_dense, le nombre d'epochs et la taille de batch. Le modèle est mis à jour après chaque lot de 32 exemples.

In [115]:
## prediction du X_test
y_pred3 = model.predict(X_test)

150/150 [==============================] - 0s 2ms/step


In [116]:
y_pred3

array([[0.8721877 ],
       [0.93152237],
       [0.8469514 ],
       ...,
       [0.99930143],
       [0.9971643 ],
       [0.9245925 ]], dtype=float32)

In [118]:
# Convertir les prédictions en étiquettes de classe (0 ou 1)
y_pred_class = (y_pred3 > 0.5).astype(int)

In [119]:
y_pred_class

array([[1],
       [1],
       [1],
       ...,
       [1],
       [1],
       [1]])

#### Prediction du dataset submission_template

In [147]:
new2=new.drop('Nom',axis=1)

In [148]:
##pretraitement du new2
X_test3 = preprocessor.transform(new2)

In [127]:
#X_test_dense = np.array(X_test3.todense())

In [149]:
#prediction
y_pred4=model.predict(X_test3)

498/498 [==============================] - 1s 2ms/step


In [151]:
y_pred4

array([[0.99997777],
       [0.9997787 ],
       [0.9997787 ],
       ...,
       [0.99883586],
       [0.94359463],
       [0.99519926]], dtype=float32)

In [152]:
# Convertir les prédictions en étiquettes de classe (0 ou 1)
y_pred4 = (y_pred4 > 0.5).astype(int)

In [153]:
y_pred4

array([[1],
       [1],
       [1],
       ...,
       [1],
       [1],
       [1]])

In [160]:
## compter les admis et les ajournes
g=pd.DataFrame(y_pred4)
g.value_counts()

1    15296
0      634
dtype: int64

In [156]:
pre=proba[:, 0]
len(pre)

15930

In [161]:
new2['predicted']=pre

In [162]:
new2.head()

,ID,"Serie,x",GroupeAnonymat,LieuNaissance,Centre,Etablissement,Willaya,moughataa,age,predicted
0,ID_15,SN,SN09,Vassala,Lycée Vassala,Lycée Vassala,Hod Charghy,BASSEKNOU,22,6.465927e-09
1,ID_22,SN,SN09,Vassala,Lycée Vassala,Lycée Vassala,Hod Charghy,BASSEKNOU,18,6.434701e-05
2,ID_23,SN,SN09,Vassala,Lycée Vassala,Lycée Vassala,Hod Charghy,BASSEKNOU,18,6.434701e-05
3,ID_29,SN,SN09,Vassala,Lycée Vassala,Lycée Vassala,Hod Charghy,BASSEKNOU,21,6.322630e-08
4,ID_34,SN,SN09,Vassala,Lycée Vassala,Lycée Vassala,Hod Charghy,BASSEKNOU,21,6.322630e-08


In [163]:
submission2=new2[['ID','predicted']]
submission2.head()

,ID,predicted
0,ID_15,6.465927e-09
1,ID_22,6.434701e-05
2,ID_23,6.434701e-05
3,ID_29,6.322630e-08
4,ID_34,6.322630e-08


In [143]:
submission.to_csv('Rn2.csv',index=False)